In [1]:
!sudo apt-get install openslide-tools -y
!sudo apt-get install python-openslide -y
!pip install openslide-python
!pip install opencv-python

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openslide-tools is already the newest version (3.4.1+dfsg-5).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package python-openslide


In [2]:
import openslide
import numpy as np
from PIL import Image

import PIL

import openslide
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import cv2 as cv
import h5py
import os

PIL.Image.MAX_IMAGE_PIXELS = 164945368800 


In [3]:
class_id=0
slide_name=77071


SLIDE_PATH = '/home/jupyter/cadaver slide/'+str(slide_name)+'.svs'
destination_PATH = '/home/jupyter/Patch_extraction/'+str(class_id)+'/'+str(slide_name)+'/'
# os.mkdir(destination_PATH)

In [4]:
percent_white_pix = 0.95
thresh_sat = 0.05
thresh_max_pix = 500

patch_size = (512,512)
patch_prefix = str(slide_name) # I find it most useful to use the slide name


In [5]:
# def patch_selection(patch, percent_white_pix, thresh_sat, thresh_max_pix):
def patch_selection(patch):

    hsv_patch = matplotlib.colors.rgb_to_hsv(patch)
    saturation = hsv_patch[:,:,1]
    percent = np.mean(saturation < thresh_sat)

    gray = cv.cvtColor(np.array(patch), cv.COLOR_RGB2GRAY)
    sobelx = cv.Sobel(gray, cv.CV_64F, 1, 0, ksize=5)
    sobely = cv.Sobel(gray, cv.CV_64F, 0, 1, ksize=5)
    gradient_magnitude = np.sqrt(sobelx**2 + sobely**2)

    return percent <= percent_white_pix and np.sum(gradient_magnitude == 0) <= thresh_max_pix

In [6]:
slide = openslide.open_slide(SLIDE_PATH)

# determine how many tiles the slide is made up of in the x direction
valid_x = int(slide.dimensions[0] / patch_size[0])
# determine how many tiles the slide is made up of in the y direction
valid_y = int(slide.dimensions[1] / patch_size[1])

# tile all valid tile positions within the slide
for x in range(valid_x):
    for y in range(valid_y):
        tile = slide.read_region(location=(x*patch_size[0], y*patch_size[1]),size=patch_size,level=0).convert('RGB')
        is_nonwhite =patch_selection(tile)

        if is_nonwhite:
            tile = np.array(tile)
            tile = Image.fromarray(tile)
            tile.save(destination_PATH + f'{patch_prefix}-{int(x*patch_size[0])}-{int(y*patch_size[1])}_class'+str(class_id)+'.png')